<a href="https://colab.research.google.com/github/KicksxNPQ/CS114.K21.KHTN/blob/master/ML_Do_an.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ĐỀ TÀI: NHẬN DIỆN XE MÁY**

1. Mô tả bài toán
- Input: Một hình ảnh chiếc xe
- Output: Cho biết chiếc xe đó có phải xe máy hay không

2. Mô tả về bộ dữ liệu
- Cách thức xây dựng bộ dữ liệu: tìm kiếm trên mạng, thu thập hình ảnh từ bạn bè.
- Số lượng: 500 ảnh
- Độ đa dạng: chỉ tập trung về xe máy.
- Các thao tác tiền xử lý dữ liệu: 
> Thu thập dữ liệu
> Gắn nhãn dữ liệu

- Phân chia dữ liệu: 
> Train/Dev/Test: 8/1/1

3. Mô tả về đặc trưng
+ Số lượng bánh xe

4. Mô tả về thuật toán máy học
- Sử dụng thuật toán Faster R-CNN

5. Cài đặt, tinh chỉnh tham số

6. Đánh giá kết quả, kết luận